In [43]:
# 特征共产鞥

In [44]:
# 导入工具包
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import gc
import copy

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


# 加载数据
train_data_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/train_format1.csv'
test_data_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/test_format1.csv'
user_info_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/user_info_format1.csv'
user_log_path = r'E:/DataSet/Tianchi/repeatPurchase/data_format1/user_log_format1.csv'

train_data = pd.read_csv(train_data_path, sep=',', encoding='utf-8')
test_data = pd.read_csv(test_data_path, sep=',', encoding='utf-8')
user_info = pd.read_csv(user_info_path, sep=',', encoding='utf-8')
user_log = pd.read_csv(user_log_path, sep=',', encoding='utf-8')

In [45]:
## 定义统计函数
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

# 数据唯一值的总数
def nunique_(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1
    
    
# 统计数据最大值
def max_(x):
    try:
        return np.max([float(i) for i in x.split(' ')])
    except:
        return -1
    
# 统计数据最小值
def min_(x):
    try:
        return np.min([float(i) for i in x.split(' ')])
    except:
        return -1
    
# 统计数据标准差
def std_(x):
    try:
        return np.std([flaot(i) for i in x.split(' ')])
    except:
        return -1
    
# 统计数据中topN的函数
def most_n(x, n):
    try:
        # [('709909', 17)] =》[0]：id
        return Counter(x.split(' ')).most_comon(n)[n-1][0]
    except:
        return -1
    
def most_n_cnt(x, n):
    try:
        # [('709909', 17)] =》[1]：次数
        return Counter(x.split(' ')).most_comon(n)[n-1][1]
    except:
        return -1

In [46]:
# 用户特征统计函数
def user_cnt(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data


def user_nunique(df_data,single_col, name):
    df_data[name] = df_data[single_col].apply(nunique_)
    return df_data

def user_max(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(max_)
    return df_data

def user_min(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(min_)
    return df_data

def user_std(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(std_)
    return df_data

def user_most_n(df_data, single_col, name, n=1):
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data


def user_most_n_cnt(df_data, single_col, name, n=1):
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data


In [47]:
# 定义内存压缩方法，（实际上就是用占内存少的数据类型来存储数据）
def reduce_mem_usage(df: pd.DataFrame, verbose=True) -> pd.DataFrame:
    start_mem = df.memory_usage().sum() / 1024**2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    for col in df.columns:
        col_type = df[col].dtypes
        if (col_type in numerics):
            col_min = df[col].min()
            col_max = df[col].max()
            if (str(col_type)[:3] == 'int'):
                if (col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (col_min > np.iinfo(np.int16).min and col_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (col_min > np.iinfo(np.int32).min and col_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (col_min > np.iinfo(np.int64).min and col_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if col_min > np.finfo(np.float16).min and col_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif col_min > np.finfo(np.float32).min and col_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_men = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is : {:.2f} MB'.format(end_men))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_men) / start_mem))
    return df
    

In [48]:
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)
user_info = reduce_mem_usage(user_info)
user_log = reduce_mem_usage(user_log)

Memory usage after optimization is : 1.74 MB
Decreased by 70.8%
Memory usage after optimization is : 3.49 MB
Decreased by 41.7%
Memory usage after optimization is : 3.24 MB
Decreased by 66.7%
Memory usage after optimization is : 890.48 MB
Decreased by 69.6%


In [49]:
# 查看压缩后的数据信息
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int32
 1   merchant_id  260864 non-null  int16
 2   label        260864 non-null  int8 
dtypes: int16(1), int32(1), int8(1)
memory usage: 1.7 MB


##  数据处理

In [50]:
# 合并用户细腻
all_data = train_data.append(test_data)
all_data = all_data.merge(user_info, on=['user_id'], how='left')
del train_data, test_data, user_info
gc.collect()


210

In [51]:
# 用户日志数据进行排序
user_log = user_log.sort_values(['user_id', 'time_stamp'])
# user_log.head(20)

In [52]:
# 合并数据 对每个用户的所有字段都进行合并
list_join_func = lambda x: ' '.join([str(i) for i in x])
agg_dict = {
    'item_id': list_join_func,
    'cat_id': list_join_func,
    'seller_id': list_join_func,
    'brand_id': list_join_func,
    'time_stamp': list_join_func,
    'action_type': list_join_func,
}
rename_dict = {
    'item_id': 'item_path',
    'cat_id': 'cat_path',
    'brand_id': 'brand_path',
    'seller_id': 'seller_path',
    'time_stamp': 'time_stamp_path',
    'action_type': 'action_type_path',
}


def merge_list(df_ID, join_columns, df_data, agg_dict, rename_dict):
    df_data = df_data.groupby(join_columns).agg(agg_dict).reset_index().rename(columns=rename_dict)
    df_ID = df_ID.merge(df_data, on=join_columns, how='left')
    return df_ID

In [53]:
# 处理效果验证
# all_data_1000 = all_data.iloc[:1000, :]
# user_log_1000 = user_log.iloc[:1000, :]
# user_log_1000.groupby(['user_id']).agg(agg_dict).reset_index().rename(columns=rename_dict).head(100)

In [54]:
all_data = merge_list(all_data, 'user_id', user_log, agg_dict, rename_dict)

In [55]:
del user_log
gc.collect()

100

In [60]:
"""
 提取基本特征
"""

all_data_test = all_data.head(2000)
# all_data_test = all_data

# 总次数
all_data_test = user_cnt(all_data_test, 'seller_path', 'user_cnt')
# 不同店铺个数
all_data_test = user_nunique(all_data_test, 'seller_path', 'seller_nunique')
# 不同品类个数
all_data_test = user_nunique(all_data_test, 'cat_path', 'cat_nunique')
# 不同品牌个数
all_data_test = user_nunique(all_data_test, 'brand_path', 'brand_nunique')
# 活跃天数
all_data_test = user_nunique(all_data_test, 'time_stamp_path', 'tiem_stamp_nunique')
# 不同用户行为种类
all_data_test = user_nunique(all_data_test, 'action_type_path', 'action_type_nunique')
# 最晚时间
all_data_test = user_max(all_data_test, 'action_type_path', 'time_stamp_max')
# 最早时间
all_data_test = user_min(all_data_test, 'action_type_path', 'time_stamp_min')
# 活跃天数方差
all_data_test = user_std(all_data_test, 'action_type_path', 'time_stamp_std')
# 最早时间和最晚相差天数
all_data_test['time_stamp_range'] = all_data_test['time_stamp_max'] - all_data_test['time_stamp_min']
# 用户最喜欢的店铺
all_data_test = user_most_n(all_data_test, 'seller_path', 'seller_most_1', n=1)
# 最喜欢的类目
all_data_test = user_most_n(all_data_test, 'cat_path', 'cat_most_1', n=1)
# 最喜欢的品牌
all_data_test = user_most_n(all_data_test, 'brand_path', 'brand_most_1', n=1)
# 最常见的行为动作
all_data_test = user_most_n(all_data_test, 'action_type_path', 'action_type_1', n=1)
# 用户最喜欢店铺的行为次数
all_data_test = user_most_n_cnt(all_data_test, 'seller_path', 'seller_most_1_cnt', n=1)
# 用户最喜欢类目的行为次数
all_data_test = user_most_n_cnt(all_data_test, 'cat_path', 'cat_most_1_cnt', n=1)
# 用户最喜欢品牌的行为次数
all_data_test = user_most_n_cnt(all_data_test, 'brand_path', 'brand_most_1_cnt', n=1)
# 用户最常见行为的次数
all_data_test = user_most_n_cnt(all_data_test, 'action_type_path', 'action_type_path_1_cnt', n=1)


In [66]:
# 对用户特征进行统计 对点击、加购、购买、收藏分开统计
def col_cnt_(df_data, columns_list, action_type):
    

0    581818 879005 581818 581818 1011673 52343 2773...
Name: item_path, dtype: object


[('709909', 17)]

['581818', '879005', '581818', '581818', '1011673', '52343', '277305', '1011348', '929296', '109785', '1011348', '581818', '338097', '1011348', '338097', '135951', '821072', '687352', '890687', '135951', '1011348', '338097', '1011348', '342730', '135951', '863054', '968989', '1011348', '729897', '677766', '516904', '179209', '179209', '325405', '179209', '325405', '733809', '413276', '862164', '901624', '901624', '901624', '901624', '901624', '198858', '901624', '901624', '975576', '1071692', '1071692', '1071692', '595685', '595685', '549726', '894551', '664341', '664341', '1071692', '974447', '1071692', '996061', '794535', '552780', '1071692', '166162', '783686', '1040019', '1107125', '53580', '783686', '783686', '783686', '1030862', '738000', '53580', '258378', '375318', '425328', '783686', '783686', '783686', '783686', '611609', '611609', '611609', '611609', '1066192', '611609', '1066192', '516503', '209475', '933626', '272116', '68555', '122852', '630464', '101527', '272116', '2721